In [3]:
import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from torchvision import transforms
from PIL import Image
import os
import matplotlib.pyplot as plt
import glob


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
image_dir = "/content/drive/MyDrive/image/xl-base-1.0" if os.path.exists("/content/drive/MyDrive") else "/home/ccl/Code/s1290114/IQA/AI_IQA/image/sd15_lcm"
dir_name = os.path.basename(image_dir)
model_name = "/content/drive/MyDrive/meta_iqa_model.pth" if os.path.exists("/content/drive/MyDrive") else "./meta_iqa_model.pth"
output_csv = f"/content/drive/MyDrive/csv/{dir_name}.csv" if os.path.exists("/content/drive/MyDrive") else f"./csv/{dir_name}.csv"

In [6]:
class MetaIQAPredictor(nn.Module):
    def __init__(self, input_size=3*512*512, hidden_size=1024, output_size=512):
        super(MetaIQAPredictor, self).__init__()
        self.model = nn.Sequential(
          nn.Linear(input_size, hidden_size),
          nn.LeakyReLU(),  # LeakyReLUで勾配消失問題を軽減
          nn.Dropout(0.3),
          nn.Linear(hidden_size, output_size),
          nn.LeakyReLU(),
          nn.Linear(output_size, 1)
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)  # 画像をフラット化
        return self.model(x)


In [7]:
def evaluate_images_and_save_csv(model, dataset, device, output_csv):
    model.eval()
    results = []
    with torch.no_grad():
        for img, img_name in dataset:
            img = img.unsqueeze(0).to(device)  # バッチ次元を追加してデバイスに移動
            predicted_score = model(img).item()  # 推定されたスコアを取得

            # 推定結果を表示
            print(f"Image: {img_name}, Predicted Score: {predicted_score}")

            # 結果をリストに追加
            results.append({
                "image_name": img_name,
                "predicted_score": predicted_score
            })

    result_df = pd.DataFrame(results)
    result_df.to_csv(output_csv, index=False)
    print(f"Results saved to {output_csv}")


In [8]:
class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.image_paths = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith(('png', 'jpg', 'jpeg'))]
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img = Image.open(img_path).convert("RGB")  # 画像をRGBとして読み込み
        img_name = os.path.basename(img_path)  # 画像ファイル名を取得

        if self.transform:
            img = self.transform(img)  # 必要に応じて画像を変換

        return img, img_name


In [9]:
transform = transforms.Compose([
        transforms.Resize((512, 512)),  # サイズを正しくリサイズ
        transforms.ToTensor(),  # テンソルに変換
])
input_size = 3*512*512
hidden_size = 1024
output_size = 512
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MetaIQAPredictor(input_size=input_size, hidden_size=hidden_size, output_size=output_size).to(device)
model.load_state_dict(torch.load(model_name, map_location=device))
dataset =  ImageDataset(image_dir, transform=transform)

<ipython-input-9-dffc6ec1e556>:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_name, map_location=device))


In [18]:
evaluate_images_and_save_csv(model, dataset, device, output_csv)

Image: 010.png, Predicted Score: 0.51494961977005
Image: 009.png, Predicted Score: 0.43974989652633667
Image: 006.png, Predicted Score: 0.5474330186843872
Image: 016.png, Predicted Score: 0.4518274664878845
Image: 001.png, Predicted Score: 0.5629040598869324
Image: 019.png, Predicted Score: 0.46618688106536865
Image: 018.png, Predicted Score: 0.3882659673690796
Image: 015.png, Predicted Score: 0.5263329148292542
Image: 011.png, Predicted Score: 0.464611291885376
Image: 004.png, Predicted Score: 0.4991381764411926
Image: 012.png, Predicted Score: 0.4767974615097046
Image: 005.png, Predicted Score: 0.6255430579185486
Image: 020.png, Predicted Score: 0.4201122522354126
Image: 013.png, Predicted Score: 0.4458591341972351
Image: 003.png, Predicted Score: 0.40594106912612915
Image: 007.png, Predicted Score: 0.6400365233421326
Image: 017.png, Predicted Score: 0.5570768713951111
Image: 014.png, Predicted Score: 0.6228225827217102
Image: 008.png, Predicted Score: 0.450619101524353
Image: 002.pn